In [23]:
import xarray as xr
ds = xr.open_dataset('/home/adboer/dlwp-benchmark/src/dlwpbench/outputs/modunet_inverted_32B_COMP_check/evaluation/inits.nc')
ds

<xarray.Dataset> Size: 5MB
Dimensions:    (sample: 204, lat: 32, lon: 64, time: 15)
Coordinates:
  * sample     (sample) datetime64[ns] 2kB 2017-01-02 2017-01-05 ... 2018-12-13
  * lat        (lat) float32 128B -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
  * lon        (lon) float32 256B 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
  * time       (time) timedelta64[ns] 120B 1 days 2 days ... 14 days 15 days
Data variables:
    msl        (sample, lat, lon) float32 2MB ...
    stream250  (sample, lat, lon) float32 2MB ...
    stream500  (sample, lat, lon) float32 2MB ...

In [44]:
import xarray as xr
import numpy as np
from datetime import timedelta

# Create a new dataset with updated coordinates
new_sample_coord = ds.sample.values
new_ds = xr.Dataset(
    coords={
        'sample': new_sample_coord,
        'time': new_time_coord,
        'lat': ds.lat,
        'lon': ds.lon
    }
)

# Create new dates for each sample
new_dates_list = []
for date in ds.sample.values:
    new_dates = [np.datetime64(date) + np.timedelta64(i, 'D') for i in range(1, 16)]
    new_dates_list.append(new_dates)
    # assign this cooridinate to the sample

new_ds = new_ds.assign_coords(forecast_date=xr.DataArray(new_dates_list, dims=('sample', 'time')))


new_ds


# now load a climatology for msl - with dimensions: month, lat, lon. i want to create a new variable in the new_ds which stores the climatology value for each forecast date

In [75]:
climatology = xr.open_dataset("/home/adboer/dlwp-benchmark/src/dlwpbench/data/netcdf/climatology_1981-2010/msl/climatology_msl.nc")


In [76]:
import xarray as xr
import numpy as np

# Assuming new_ds and climatology are already loaded

# Extract month from forecast_date
forecast_months = new_ds.forecast_date.dt.month

# Create a month coordinate in the climatology dataset if it doesn't exist
if 'month' not in climatology.coords:
    climatology = climatology.assign_coords(month=('time', climatology.time.dt.month))


# Select the relevant months from the climatology
climatology_subset = climatology.sel(month=np.unique(forecast_months.values.ravel()))

# Broadcast the climatology to match the shape of new_ds
broadcasted_climatology = climatology_subset.broadcast_like(new_ds.isel(sample=0, time=0))

# Assign the broadcasted climatology to new_ds
new_ds['msl'] = broadcasted_climatology.msl.sel(month=forecast_months)

print(new_ds)


<xarray.Dataset> Size: 351MB
Dimensions:             (lat: 32, lon: 64, sample: 204, time: 15, level: 2)
Coordinates:
  * lat                 (lat) float32 128B -87.19 -81.56 -75.94 ... 81.56 87.19
  * lon                 (lon) float32 256B 0.0 5.625 11.25 ... 343.1 348.8 354.4
  * sample              (sample) datetime64[ns] 2kB 2017-01-02 ... 2018-12-13
  * time                (time) int64 120B 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
    forecast_date       (sample, time) datetime64[ns] 24kB 2017-01-03 ... 201...
    weights             (lon, lat) float64 16kB ...
    month               (sample, time) int64 24kB 1 1 1 1 1 1 ... 12 12 12 12 12
  * level               (level) int64 16B 250 500
Data variables:
    msl_climatology     (lat, lon, sample, time) float64 50MB 1.005e+05 ... 1...
    stream_climatology  (lat, lon, sample, time, level) float64 100MB 1.32e+0...
    stream              (lat, lon, level, sample, time) float64 100MB 1.32e+0...
    msl                 (lat, lon, level, 

In [79]:
new_ds = new_ds.drop_vars('stream_climatology')
new_ds

<xarray.Dataset> Size: 150MB
Dimensions:          (lat: 32, lon: 64, sample: 204, time: 15, level: 2)
Coordinates:
  * lat              (lat) float32 128B -87.19 -81.56 -75.94 ... 81.56 87.19
  * lon              (lon) float32 256B 0.0 5.625 11.25 ... 343.1 348.8 354.4
  * sample           (sample) datetime64[ns] 2kB 2017-01-02 ... 2018-12-13
  * time             (time) int64 120B 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
    forecast_date    (sample, time) datetime64[ns] 24kB 2017-01-03 ... 2018-1...
    weights          (lon, lat) float64 16kB ...
    month            (sample, time) int64 24kB 1 1 1 1 1 1 ... 12 12 12 12 12 12
  * level            (level) int64 16B 250 500
Data variables:
    msl_climatology  (lat, lon, sample, time) float64 50MB 1.005e+05 ... 1.01...
    stream           (lat, lon, level, sample, time) float64 100MB 1.32e+08 ....

In [80]:
new_ds = new_ds.rename({'msl_climatology': 'msl'})
new_ds

<xarray.Dataset> Size: 150MB
Dimensions:        (lat: 32, lon: 64, sample: 204, time: 15, level: 2)
Coordinates:
  * lat            (lat) float32 128B -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
  * lon            (lon) float32 256B 0.0 5.625 11.25 ... 343.1 348.8 354.4
  * sample         (sample) datetime64[ns] 2kB 2017-01-02 ... 2018-12-13
  * time           (time) int64 120B 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
    forecast_date  (sample, time) datetime64[ns] 24kB 2017-01-03 ... 2018-12-28
    weights        (lon, lat) float64 16kB ...
    month          (sample, time) int64 24kB 1 1 1 1 1 1 1 ... 12 12 12 12 12 12
  * level          (level) int64 16B 250 500
Data variables:
    msl            (lat, lon, sample, time) float64 50MB 1.005e+05 ... 1.013e+05
    stream         (lat, lon, level, sample, time) float64 100MB 1.32e+08 ......

In [81]:
new_ds.to_netcdf("FINAL_climatology.nc")

In [77]:
new_ds['msl'].isel(sample=9).isel(time=0) #.shape

<xarray.DataArray 'msl' (lat: 32, lon: 64, level: 2)> Size: 33kB
array([[[100676.58884999, 100676.58884999],
        [100709.16051299, 100709.16051299],
        [100753.74457532, 100753.74457532],
        ...,
        [100520.23662884, 100520.23662884],
        [100565.09487328, 100565.09487328],
        [100622.63568425, 100622.63568425]],

       [[100071.42769086, 100071.42769086],
        [100114.74311684, 100114.74311684],
        [100267.68049709, 100267.68049709],
        ...,
        [ 99660.19419323,  99660.19419323],
        [ 99839.91608681,  99839.91608681],
        [ 99998.77956189,  99998.77956189]],

       [[ 99576.88537142,  99576.88537142],
        [ 99680.70173981,  99680.70173981],
        [ 99824.01722826,  99824.01722826],
        ...,
...
        ...,
        [101129.76114591, 101129.76114591],
        [100876.18275409, 100876.18275409],
        [100646.16946191, 100646.16946191]],

       [[101062.04070142, 101062.04070142],
        [100994.62401105, 100994.62401105],
        [100955.42931549, 100955.42931549],
        ...,
        [101354.30305142, 101354.30305142],
        [101256.97093576, 101256.97093576],
        [101154.65549255, 101154.65549255]],

       [[101376.90907666, 101376.90907666],
        [101371.65025476, 101371.65025476],
        [101369.28622171, 101369.28622171],
        ...,
        [101402.18558628, 101402.18558628],
        [101391.83483699, 101391.83483699],
        [101383.69432529, 101383.69432529]]])
Coordinates:
  * lat            (lat) float32 128B -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
  * lon            (lon) float32 256B 0.0 5.625 11.25 ... 343.1 348.8 354.4
    sample         datetime64[ns] 8B 2017-02-02
    time           int64 8B 1
    forecast_date  datetime64[ns] 8B 2017-02-03
    weights        (lon, lat) float64 16kB ...
    month          int64 8B 2
  * level          (level) int64 16B 250 500
Attributes:
    cell_methods:   time: mean
    long_name:      Mean sea level pressure
    standard_name:  air_pressure_at_mean_sea_level
    units:          Pa

In [45]:
# Extract month from forecast_date
forecast_months = new_ds.forecast_date.dt.month

# Create a new variable for climatology in new_ds
new_ds['msl_climatology'] = xr.apply_ufunc(
    lambda month, lat, lon: climatology.msl.sel(month=month, lat=lat, lon=lon, method='nearest'),
    forecast_months,
    new_ds.lat,
    new_ds.lon,
    input_core_dims=[[], [], []],
    output_core_dims=[[]],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float]
)

KeyboardInterrupt: 

In [ ]:
# group the dataset by ds.groupby("sample.month"), load the climaology dataframe (which has 12 months)
# and set the values of ds.msl equal to the climatology value (assign this)

In [20]:
df = ds.groupby("sample.month") # set this equal to climatology
df

<DatasetGroupBy, grouped over 1 grouper(s), 12 groups in total:
    'month': 12/12 groups present with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12>

In [16]:
ds=  ds.groupby("time.month").mean("time")

In [17]:
ds.to_netcdf('CLIM_STREAM.nc')

In [9]:
data_rolling = ds.rolling(
                dim={"time": 25},  
                min_periods=1,
                center=True,
            ).mean()

In [10]:
data_rolling.to_netcdf("Climatology_msl_FINAL.nc")